# Collecting and preparing data

In [1]:
import os, json, requests
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import cbsodata
from datetime import date, timedelta
import tabula
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re
from tqdm.notebook import tqdm
from datetime import datetime as dt
from itertools import groupby

In [2]:
today = date.today()
pwd = os.getcwd()
if not pwd.endswith('data'):
    pwd = pwd + '/data'

csv_dir = pwd + '/csvs/'
sources = pwd + '/sources.json'
rivm_pdf_data_loc = pwd + '/pdfs/'

with open(sources, 'r') as json_file:
        resources = json.load(json_file)

def cleanup(df):
    for c in df.columns:
        if 'ersion' in c.lower():
            df.drop(columns=c, inplace=True)
    return df

In [3]:
def data_weekreports(main_url):
    def columns(l):
        def ll(l):
            return -l.count('')

        if 'http' in l:
            l = l[ll(l) + 20:]

        href.append(l)
        period.append(pd.Period(l[ll(l) + 22 : ll(l) + 29],"M"))
        names.append(l[ll(l) + 65 : -4])
        datetime.append(pd.to_datetime(l[ll(l) + 65 : ll(l) + 73], format="%Y%m%d"))
        filename.append(l[ll(l) + 65:])
        url.append('https://www.rivm.nl'+l)
    
    def extract_tabel(pdf,date):
        def check_tabel(value):
            for key in checks:
                if checks[key]:
                    pass
                elif isinstance(value,str) and key in value:
                    checks[key] = True
                    
        def create_agegroup_tabel(tabel, n):
            #print(tabel)
            if checks['Leeftijdsgroep'] and checks['fgelopen']:
                category = 'Leeftijd'
            elif checks['Groep'] and checks['fgelopen'] and checks['Kinderen']:
                category = 'Groep'
            elif checks['Groep'] and checks['Kinderen']:
                category = 'Groep'
            elif checks['Leeftijdsgroep']:
                category = 'Leeftijd'
                n+=1
            else:
                category = ''
            #print(checks)
            #print(category)
            csv = pdf[:-4]+'-'+category+'.csv'

            if category != '':
                #tabel.dropna(inplace=True)
                if not tabel.empty:
                    #print('cat='+category)
                    first_column = tabel.columns[0]
                    agegroup_tabel = tabel.loc[(tabel[first_column].str.count('-') > 0)]                
                    if category.startswith('Leeftijd'):
                        agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.find('+') > 0)])
                        if len(tabel.columns) > 1:
                            second_column = tabel.columns[1]
                        else:
                            second_column = 'second_column'
                        agegroup_tabel.rename(columns={first_column:'Leeftijdsgroep',second_column:date}, inplace=True)
                    elif category == 'Groep':
                        agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.count('fgelopen') > 0)])
                        agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.count('Groep') > 0)])
                    agegroup_tabel = agegroup_tabel.sort_index()
                    agegroup_tabel['Date'] = date
                    agegroup_tabel.to_csv(csv, index=False)
                    #print('CSV CREATED')
                    #print(agegroup_tabel)
                    return True

        #print('extracting '+pdf)
        #print(pdf)
        found_pages = []
        object = PyPDF2.PdfFileReader(pdf)
        NumPages = object.getNumPages()
        String = "SARS-COV-2.*TESTEN.*AFGENOMEN.*DOOR.*DE.*GGD"

        for i in tqdm(range(0, NumPages)):
            PageObj = object.getPage(i)
            Text = PageObj.extractText()
            if re.search(String,Text):
                found_pages.append(i)

        if len(found_pages) == 0:
            print("NO CHAPTER FOUND in "+str(pdf))
        else:
            tabels = tabula.read_pdf(input_path=pdf, pages=found_pages, guess=True, stream=False, multiple_tables=True)
            csv_created = None
            n = 1
            for tabel in tabels:
                #print(tabel)
                checks = {
                    'Leeftijdsgroep': False,
                    'fgelopen': False,
                    'Groep': False,
                    'Kinderen': False}
                for column in tabel:
                    #print(column)
                    if column.startswith("Unnamed"):
                        newname = ""
                        if tabel.shape[0] > 1:
                            newname = str(tabel.loc[1,column])
                            if tabel.loc[0,column] == np.NaN:
                                newname = newname + " "
                        #print(newname)
                        tabel.rename(columns={column:newname}, inplace=True)
                        column = newname
                    check_tabel(column)
                    for i in tabel[column].values:
                        #print(i)
                        check_tabel(i)
                #print(checks)
                create_agegroup_tabel(tabel, n)
    
    url, href, period, names, datetime, week, filename = [], [], [], [], [], [], []
    response = requests.get(main_url)
    soup = BeautifulSoup(response.text, 'html.parser') 
    aa = soup.find_all('a')
    for a in aa:
        link = a.get('href')
        if link == None:
            pass
        elif link.endswith('.pdf'):
            columns(link)
    
    pdfs = pd.DataFrame([period, datetime, week, names, filename, href, url]).T
    pdfs = pdfs.rename(columns={0:'period',1:'date',2:'week',3:'name',4:'filename',5:'href',6:'url'})
    pdfs['week'] = pdfs.date.dt.isocalendar().week
    pdfs.sort_values(by='period', inplace=True)
    
    # download all pdfs & extract table tp csv
    #print('Checking for new weekreport')
    for row in pdfs.itertuples():
        pdf = rivm_pdf_data_loc+row.filename
        #print(pdf)
        try:
            try:
                csv = pdf[:-4]+'-Leeftijd.csv'
                os.stat(csv)
            except:
                try:
                    csv = pdf[:-4]+'-Groep.csv'
                    os.stat(csv)
                except:
                    csv = pdf[:-4]+'-dummy.csv'
                    os.stat(csv)
        except:
            try:
                os.stat(pdf)
            except:
                #print('downloading '+row.filename)
                r = requests.get(row.url, stream=True)
                with open(pdf, 'wb') as f:
                    f.write(r.content)
            finally:
                #print("extracting tabels from pdf")
                extract_tabel(pdf, row.date)

## (Down)loading data

In [4]:
print('Pulling data from sources.json:')

for org,sets in resources.items():
    for dataset,meta in sets.items():
        name = '_'.join(dataset.lower().split())
        data_loc = csv_dir + org.lower() + '_' + name + '.csv'
        updated = meta['updated']
        try:
            os.stat(data_loc)
        except:
            no_csv = True
        else:
            no_csv = False
        if no_csv or updated != str(today):
            if org == 'CBS':
                identifier = meta['code']
                pd.DataFrame(cbsodata.get_data(identifier)).to_csv(data_loc, index=False)
                #metadata = cbsodata.get_info(identifier)
            elif org == 'RIVM' and dataset == 'wekelijkse update':
                data_weekreports(meta['data'])
            else:
                if meta['data'][-4:] == '.csv':
                    pd.read_csv(meta['data']).to_csv(data_loc, index=False)
                elif meta['data'][-5:] == '.json':
                    pd.read_json(meta['data']).to_csv(data_loc, index=False)
                else:
                    print('can not handle file format: just dopwnloading file')
                    data_loc = data_loc[-4:]+'.temp'
                    print(data_loc)
                    print(meta['data'])
                    req = requests.get(meta['data'], stream=True)
                    print(req)
                    with open(data_loc[:-5], 'wb') as f:
                        f.write(req.content)  
            meta['updated'] = str(today)
        
        if org == 'RIVM' and dataset == 'wekelijkse update':
            tmp = data_loc
        else:
            globals()[name] = pd.read_csv(data_loc)
        print('(Source: '+org+')  '+ name +' @ ' + meta['meta'])

with open(sources, "w") as outfile:
    json.dump(resources, outfile, indent=4)

Pulling data from sources.json:
(Source: CBS)  deceased @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=70895ned&_theme=76
(Source: CBS)  population @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=37230ned&_theme=76
(Source: RIVM)  counts_per_municipality @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427
(Source: RIVM)  prevalence @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/097155aa-75eb-4caa-8ed3-4c6edb80467e
(Source: RIVM)  r @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
(Source: RIVM)  homes_for_elderly @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/0179dd26-7bf6-4021-857f-8623aaf8e73a
(Source: RIVM)  homes_for_disabled @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/aa28b74d-7150-47bc-a7cb-98876e93381b
(Source: RIVM)  seventy_plus_living_at_home @ https://da

## Preparing data

In [5]:
print()
print('Creating dataframes for analysis:')


Creating dataframes for analysis:


### CBS
#### Deceased

In [6]:
# Totals only
if 'ID' in deceased.columns:
    deceased = deceased.drop(columns='ID')
if 'Geslacht' in deceased.columns:
    deceased = deceased.loc[deceased['Geslacht'] == 'Totaal mannen en vrouwen',:]
    deceased.drop(columns='Geslacht', inplace=True)
deceased = deceased.loc[deceased['LeeftijdOp31December'] != 'Totaal leeftijd',:]
deceased.rename(columns={'Overledenen_1':'Deceased'}, inplace=True)

#### Deceased per week

In [7]:
# weeks only, no totals for year
deceased_week_agegroup = deceased[deceased['Perioden'].str.contains('week')]
deceased_week_agegroup['year'] = deceased_week_agegroup['Perioden'].str.slice(stop=4).astype(int)

# split Perioden into year and week
deceased_week_agegroup['week'] = deceased_week_agegroup['Perioden'].str.slice(start=10,stop=12).astype(int)
deceased_week_agegroup = deceased_week_agegroup.loc[deceased_week_agegroup.week != '']
#deceased_week_agegroup['week'] = deceased_week_agegroup['week']

# adding y,week53 to y+1,week00
deceased_week_agegroup.loc[deceased_week_agegroup.loc[:,'week'] == 53, 'year'] += 1
deceased_week_agegroup.loc[deceased_week_agegroup.loc[:,'week'] == 53, 'week'] = 0

# set Index
deceased_week_agegroup.index = pd.MultiIndex.from_frame(deceased_week_agegroup[['year','week']])
deceased_week_agegroup.drop(columns=['year','week', 'Perioden'], inplace=True)
deceased_week_agegroup = deceased_week_agegroup.rename(columns={'Overledenen_1': 'Deceased'})

print('dataframe from cbs created, per week per agegroup: deceased_week_agegroup')
deceased_week_agegroup

dataframe from cbs created, per week per agegroup: deceased_week_agegroup


LeeftijdOp31December  Deceased
year week                               
1995 0           0 tot 65 jaar      62.0
     1           0 tot 65 jaar     476.0
     2           0 tot 65 jaar     511.0
     3           0 tot 65 jaar     473.0
     4           0 tot 65 jaar     513.0
...                        ...       ...
2021 51       80 jaar of ouder    2102.0
     52       80 jaar of ouder    1463.0
2022 0        80 jaar of ouder     549.0
     1        80 jaar of ouder    1865.0
     2        80 jaar of ouder    1865.0

[4305 rows x 2 columns]

In [8]:
deceased_per_week = deceased_week_agegroup.groupby(['year','week']).sum()

print('dataframe from cbs created, per week: deceased_per_week')
deceased_per_week

dataframe from cbs created, per week: deceased_per_week


Deceased
year week          
1995 0        394.0
     1       2719.0
     2       2823.0
     3       2609.0
     4       2664.0
...             ...
2021 51      3740.0
     52      2698.0
2022 0        946.0
     1       3392.0
     2       3238.0

[1430 rows x 1 columns]

In [9]:
# years only
deceased_year_agegroup = deceased[deceased['Perioden'].str.contains('week') == False]
deceased_year_agegroup = deceased_year_agegroup.rename(columns={'Perioden': 'year'})
deceased_year_agegroup.set_index('year', inplace=True)
deceased_year_agegroup.index = pd.PeriodIndex(deceased_year_agegroup.index, freq='Y')

print('dataframe from cbs created, per year per agegroup: deceased_year_agegroup')
deceased_year_agegroup

dataframe from cbs created, per year per agegroup: deceased_year_agegroup


,LeeftijdOp31December,Deceased
year,,
1995,0 tot 65 jaar,26183.0
1996,0 tot 65 jaar,26388.0
1997,0 tot 65 jaar,25715.0
1998,0 tot 65 jaar,26031.0
1999,0 tot 65 jaar,26557.0
...,...,...
2017,80 jaar of ouder,84095.0
2018,80 jaar of ouder,85820.0
2019,80 jaar of ouder,85010.0


In [10]:
deceased_per_year = deceased_year_agegroup.groupby('year').sum()

print('dataframe from cbs created, per year: deceased_per_year')
deceased_per_year

dataframe from cbs created, per year: deceased_per_year


,Deceased
year,
1995,135675.0
1996,137561.0
1997,135783.0
1998,137482.0
1999,140487.0
2000,140527.0
2001,140377.0
2002,142355.0
2003,141936.0


### Population & Deceased

In [11]:
# dropping unnececary columns
population.drop(columns=['ID','BevolkingAanHetBeginVanDePeriode_1','TotaleVestiging_4',
       'VestigingVanuitEenAndereGemeente_5', 'Immigratie_6',
       'TotaalVertrekInclAdmCorrecties_7', 'VertrekNaarAndereGemeente_8',
       'EmigratieInclusiefAdmCorrecties_9', 'OverigeCorrecties_10',
       'Bevolkingsgroei_11', 'BevolkingsgroeiRelatief_12',
       'BevolkingsgroeiSinds1Januari_13',
       'BevolkingsgroeiSinds1JanuariRela_14'], inplace=True)
population.rename(columns={"LevendGeborenKinderen_2": "Born",
                                        "Overledenen_3": "Deceased",
                                        "BevolkingAanHetEindeVanDePeriode_15": "Population"}, inplace=True)

In [12]:
# RegioS only, no nationals
population = population[population['RegioS'] != 'Nederland']
population = population[population['RegioS'].str.contains('LD') == False]
population = population[population['RegioS'].str.contains('PV') == False]
population = population[population['RegioS'].str.contains('CR') == False]

# Preparing periodes
population['year'] = population.loc[:,'Perioden'].str.slice(stop=4)
population['month'] = population.loc[:,'Perioden'].str.slice(start=5)
population = population.loc[population['month'] != '',:]
population.loc[:,'month'].replace({
    'januari': '01',
    'februari': '02',
    'maart': '03',
    'april': '04',
    'mei': '05',
    'juni': '06',
    'juli': '07',
    'augustus': '08',
    'september': '09',
    'oktober': '10',
    'november': '11',
    'december': '12'}, inplace=True)
population['Y-M'] = population['year'] + '-' + population['month']

population_month_municipality = population.groupby(['Y-M','RegioS']).sum()
population_month_municipality.reset_index(level='RegioS', inplace=True)
population_month_municipality.index = pd.PeriodIndex(population_month_municipality.index, freq='M')

print('dataframe from cbs created, per month per municipality: population_month_municipality')
population_month_municipality

dataframe from cbs created, per month per municipality: population_month_municipality


,RegioS,Born,Deceased,Population
Y-M,,,,
2002-01,'s-Gravendeel,9.0,7.0,8792.0
2002-01,'s-Gravenhage (gemeente),556.0,469.0,458474.0
2002-01,'s-Gravenzande,28.0,16.0,19426.0
2002-01,'s-Hertogenbosch,140.0,98.0,131731.0
2002-01,Aa en Hunze,25.0,21.0,25578.0
...,...,...,...,...
2021-11,Zutphen,30.0,54.0,48372.0
2021-11,Zwartewaterland,23.0,17.0,23003.0
2021-11,Zwijndrecht,37.0,61.0,44749.0


In [13]:
population_year_municipality = population.groupby(['year','RegioS']).sum()
population_year_municipality.reset_index(level='RegioS', inplace=True)
population_year_municipality.index = pd.PeriodIndex(population_year_municipality.index, freq='Y')

print('dataframe from cbs created, per year per municipality: population_year_municipality')
population_year_municipality

dataframe from cbs created, per year per municipality: population_year_municipality


,RegioS,Born,Deceased,Population
year,,,,
2002,'s-Gravendeel,102.0,66.0,106209.0
2002,'s-Gravenhage (gemeente),6328.0,4993.0,5534760.0
2002,'s-Gravenzande,253.0,136.0,233555.0
2002,'s-Hertogenbosch,1728.0,1119.0,1584919.0
2002,Aa en Hunze,289.0,251.0,306187.0
...,...,...,...,...
2021,Zutphen,356.0,458.0,532028.0
2021,Zwartewaterland,262.0,150.0,252132.0
2021,Zwijndrecht,445.0,484.0,492483.0


In [14]:
population_per_month = population.groupby('Y-M').sum()
population_per_month.index = pd.PeriodIndex(population_per_month.index, freq='M')

print('dataframe from cbs created, per month: population_per_month')
population_per_month

dataframe from cbs created, per month: population_per_month


,Born,Deceased,Population
Y-M,,,
2002-01,17019.0,13469.0,16112900.0
2002-02,15448.0,11735.0,16118953.0
2002-03,16792.0,13281.0,16124125.0
2002-04,15995.0,11968.0,16130246.0
2002-05,16800.0,11623.0,16136807.0
...,...,...,...
2021-07,16053.0,12557.0,17515152.0
2021-08,15798.0,12726.0,17535520.0
2021-09,16086.0,12702.0,17565777.0


In [15]:
population_per_year = population.groupby('year').sum()
deceased_per_year = deceased_year_agegroup.groupby('year').sum()
population_per_year = pd.concat([deceased_per_year,population_per_year])
population_per_year.index = pd.PeriodIndex(population_per_year.index, freq='Y')

print('dataframe from cbs created, per month: population_per_year')
population_per_year

dataframe from cbs created, per month: population_per_year


,Deceased,Born,Population
year,,,
1995,135675.0,NaN,NaN
1996,137561.0,NaN,NaN
1997,135783.0,NaN,NaN
1998,137482.0,NaN,NaN
1999,140487.0,NaN,NaN
2000,140527.0,NaN,NaN
2001,140377.0,NaN,NaN
2002,142355.0,NaN,NaN
2003,141936.0,NaN,NaN


### RIVM
#### RIVM per dag

In [16]:
counts_per_municipality

,Date_of_report,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Hospital_admission,Deceased
0,2021-12-12 10:00:00,2020-02-28,GM0014,Groningen,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,0
1,2021-12-12 10:00:00,2020-02-28,GM0034,Almere,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0,0
2,2021-12-12 10:00:00,2020-02-28,GM0037,Stadskanaal,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,0
3,2021-12-12 10:00:00,2020-02-28,GM0047,Veendam,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,0
4,2021-12-12 10:00:00,2020-02-28,GM0050,Zeewolde,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
247207,2021-12-12 10:00:00,2021-12-12,NaN,NaN,Limburg,VR23,Limburg-Noord,GGD Limburg-Noord,Netwerk Acute Zorg Limburg,3,0,0
247208,2021-12-12 10:00:00,2021-12-12,NaN,NaN,Limburg,VR24,Limburg-Zuid,GGD Zuid-Limburg,Netwerk Acute Zorg Limburg,9,0,0
247209,2021-12-12 10:00:00,2021-12-12,NaN,NaN,Zuid-Holland,VR15,Haaglanden,GGD Haaglanden,Netwerk Acute Zorg West,16,1,0
247210,2021-12-12 10:00:00,2021-12-12,NaN,NaN,Fryslân,VR02,Fryslân,GGD Fryslân,Acute Zorgnetwerk Noord Nederland,6,0,0


In [17]:
prevalence = cleanup(prevalence)
prevalence = prevalence.groupby('Date').sum()
prevalence.index = pd.PeriodIndex(prevalence.index, freq='D')
rivm_per_day = prevalence

counts_per_municipality.rename(columns={'Date_of_publication':'Date',
                                        'Total_reported': 'Positive_tests',
                                        'Deceased': 'Deceased_covid'
                                       }, inplace=True)
counts_per_municipality = counts_per_municipality.groupby('Date').sum()
counts_per_municipality = counts_per_municipality[['Deceased_covid', 'Positive_tests', 'Hospital_admission']]
counts_per_municipality.index = pd.PeriodIndex(counts_per_municipality.index, freq='D')
rivm_per_day = rivm_per_day.merge(counts_per_municipality, on='Date', how='outer')

hospital_admission.rename(columns={'Date_of_statistics': 'Date',
                                   'Hospital_admission_notification': 'Hospital_admission_NICE_notification',
                                   'Hospital_admission': 'Hospital_admission_NICE'
                                  }, inplace=True)
hospital_admission = cleanup(hospital_admission)
hospital_admission = hospital_admission.groupby('Date').sum()
hospital_admission = hospital_admission[['Hospital_admission_NICE','Hospital_admission_NICE_notification']]
hospital_admission.index = pd.PeriodIndex(hospital_admission.index, freq='D')
rivm_per_day = rivm_per_day.merge(hospital_admission, on='Date', how='outer')

ic_admission.rename(columns={'Date_of_statistics': 'Date'}, inplace=True)
ic_admission = cleanup(ic_admission)
ic_admission = ic_admission.groupby('Date').sum()
ic_admission = ic_admission[['IC_admission','IC_admission_notification']]
ic_admission.index = pd.PeriodIndex(ic_admission.index, freq='D')
rivm_per_day = rivm_per_day.merge(ic_admission, on='Date', how='outer')

hospital_occupation.rename(columns={'Datum': 'Date'}, inplace=True)
hospital_occupation['Date'] = pd.to_datetime(hospital_occupation['Date'], format='%d-%m-%Y')
hospital_occupation['IC_Bedden_TOTAAL'] = hospital_occupation['IC_Bedden_COVID']+hospital_occupation['IC_Bedden_Non_COVID']
hospital_occupation = hospital_occupation.groupby('Date').sum()
hospital_occupation.index = pd.PeriodIndex(hospital_occupation.index, freq='D')
rivm_per_day = rivm_per_day.merge(hospital_occupation, on='Date', how='outer')

r = cleanup(r)
#r.rename(columns={'population': 'source_for_r'}, inplace=True)
r = r.groupby('Date').sum()
#r = r.groupby(['Date','source_for_r']).sum()
#r = r.reset_index(level='source_for_r')
r.index = pd.PeriodIndex(r.index, freq='D')
r = r.loc[r.index <= pd.Period(today-timedelta(days=14), freq='D')]
rivm_per_day = rivm_per_day.merge(r, on='Date', how='outer')

homes_for_elderly.rename(columns={
    'Date_of_statistic_reported': 'Date',
    'Total_cases_reported': 'Positive_tests_elderly_homes',
    'Total_deceased_reported': 'Deceased_elderly_homes',
    'Total_new_infected_locations_reported': 'new_infected_elderly_homes',
    'Total_infected_locations_reported': 'balance_infected_elderly_homes'
    }, inplace=True)
homes_for_elderly = homes_for_elderly.groupby('Date').sum()
homes_for_elderly.index = pd.PeriodIndex(homes_for_elderly.index, freq='D')
homes_for_elderly = homes_for_elderly.loc['2020':]
rivm_per_day = rivm_per_day.merge(homes_for_elderly, on='Date', how='outer')


## Not yet implemented
# homes_for_disabled
# seventy_plus_living_at_home
# sewage
# support_and_behavior
# general_practioner

## STEP FOR ADDING
# rename columns if nececary
# cleanup if nececary
# groupby('Date').sum()
# re-order columns in nececeray
# apply periodIndex(df, freq='D')

#rivm_per_day = rivm_per_day.astype(int)

print('dataframe for RIVM data created, per day: rivm_per_day')
rivm_per_day

dataframe for RIVM data created, per day: rivm_per_day


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Nieuwe_Opnames_COVID,Kliniek_Nieuwe_Opnames_COVID,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-17,693.0,1074.0,1462.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.93,4.00,NaN,NaN,NaN,NaN
2020-02-18,808.0,1232.0,1653.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,1.74,4.00,NaN,NaN,NaN,NaN
2020-02-19,989.0,1461.0,1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.59,2.83,NaN,NaN,NaN,NaN
2020-02-20,1188.0,1718.0,2244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.83,1.74,2.83,NaN,NaN,NaN,NaN
2020-02-21,1466.0,2071.0,2667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.67,2.03,3.67,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,NaN,NaN,NaN,NaN,NaN,NaN,138.0,114.0,14.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,119.0,4.0,21.0,510.0
2022-01-20,NaN,NaN,NaN,NaN,NaN,NaN,114.0,151.0,10.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,124.0,1.0,28.0,525.0
2022-01-21,NaN,NaN,NaN,NaN,NaN,NaN,67.0,145.0,7.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,82.0,1.0,28.0,542.0


In [18]:
# estimating missing prevalance numbers since july 2021
# https://www.rivm.nl/coronavirus-covid-19/rekenmodellen
rivm_per_day['PREV_NAN'] = rivm_per_day['prev_avg'] / (rivm_per_day['Hospital_admission_NICE'].rolling(7).sum()/7)
factor = rivm_per_day.loc['2020-11':'2021-06-28','PREV_NAN'].dropna().mean()
rivm_per_day.drop(columns='PREV_NAN', inplace=True)

n=0
for i in rivm_per_day['prev_avg']:
    if np.isnan(i) or i == 0.0:
        rivm_per_day.iloc[n,1] = factor * rivm_per_day.iloc[n,6]
    n+=1

rivm_per_day.loc['2020-06-29':,'prev_avg'] = rivm_per_day.loc['2020-06-29':,'prev_avg'].rolling(7).mean()

#### RIVM per week

In [19]:
rivm_per_day['year'] = rivm_per_day.index.year
rivm_per_day['week'] = rivm_per_day.index.strftime('%W').astype(int)
rivm_per_week = rivm_per_day.groupby(by=['year', 'week']).sum()

print('dataframe for RIVM data created: rivm_per_week')
rivm_per_week

dataframe for RIVM data created: rivm_per_week


prev_low       prev_avg    prev_up  Deceased_covid  Positive_tests  \
year week                                                                       
2020 7       9143.0   12981.000000    16854.0             0.0             0.0   
     8      43236.0   53641.000000    63719.0             0.0             5.0   
     9     174623.0  209911.000000   240870.0             2.0           158.0   
     10    395330.0  494058.000000   582199.0            15.0           946.0   
     11    726956.0  907972.000000  1075563.0           162.0          3076.0   
...             ...            ...        ...             ...             ...   
2021 52         0.0  478609.508399        0.0             0.0             0.0   
2022 0          0.0  182405.314442        0.0             0.0             0.0   
     1          0.0  572265.469755        0.0             0.0             0.0   
     2          0.0  494964.869298        0.0             0.0             0.0   
     3          0.0  368598.186145        0.0             0.0             0.0   

           Hospital_admission  Hospital_admission_NICE  \
year week                                                
2020 7                    0.0                      0.0   
     8                    2.0                     17.0   
     9                   29.0                     95.0   
     10                 129.0                    433.0   
     11                 816.0                   1690.0   
...                       ...                      ...   
2021 52                   0.0                    835.0   
2022 0                    0.0                    217.0   
     1                    0.0                    951.0   
     2                    0.0                    732.0   
     3                    0.0                    547.0   

           Hospital_admission_NICE_notification  IC_admission  \
year week                                                       
2020 7                                      0.0           0.0   
     8                                      0.0           2.0   
     9                                      0.0          18.0   
     10                                     0.0         111.0   
     11                                     0.0         426.0   
...                                         ...           ...   
2021 52                                   954.0         125.0   
2022 0                                    170.0          45.0   
     1                                    943.0         146.0   
     2                                    792.0          97.0   
     3                                    783.0          49.0   

           IC_admission_notification  ...  IC_Nieuwe_Opnames_COVID  \
year week                             ...                            
2020 7                           0.0  ...                      0.0   
     8                           0.0  ...                      0.0   
     9                           0.0  ...                      0.0   
     10                          0.0  ...                      0.0   
     11                          0.0  ...                      0.0   
...                              ...  ...                      ...   
2021 52                        147.0  ...                      0.0   
2022 0                          38.0  ...                      0.0   
     1                         148.0  ...                      0.0   
     2                         113.0  ...                      0.0   
     3                          63.0  ...                      0.0   

           Kliniek_Nieuwe_Opnames_COVID  IC_Bedden_TOTAAL  Rt_low  Rt_avg  \
year week                                                                   
2020 7                              0.0               0.0    3.67   13.38   
     8                              0.0               0.0    9.68   14.39   
     9                              0.0               0.0   10.79   12.90   
     10                             0.0               0.0   

In [20]:
#print('dataframe for RIVM data created, per week per municipality: ririvm_week_municipalityvm_per_week')
#rivm_week_municipality

#### RIVM per month

In [21]:
rivm_per_day['Y-M'] = rivm_per_day.index.strftime('%Y') + '-' + rivm_per_day.index.strftime('%m')
rivm_per_month = rivm_per_day.groupby('Y-M').sum()
rivm_per_day.drop(columns='Y-M', inplace=True)
rivm_per_month.index = pd.PeriodIndex(rivm_per_month.index, freq='M')
rivm_per_month = rivm_per_month[rivm_per_month.index < population_per_month.index.max()]

print('dataframe for RIVM data created, per month: rivm_per_month')
rivm_per_month

dataframe for RIVM data created, per month: rivm_per_month


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
Y-M,,,,,,,,,,,,,,,,,,,,,
2020-02,41197.0,5.317100e+04,64978.0,0.0,2.0,1.0,9.0,0.0,1.0,0.0,...,0.0,11.91,25.94,43.07,0.0,0.0,0.0,0.0,26260,97
2020-03,2334676.0,3.042902e+06,3716756.0,1018.0,12447.0,4636.0,6372.0,4124.0,1545.0,1403.0,...,0.0,31.80,36.69,41.76,1794.0,397.0,582.0,4620.0,62620,328
2020-04,1319824.0,1.944858e+06,2613234.0,3747.0,26610.0,6056.0,4843.0,6020.0,1113.0,1192.0,...,17913.0,15.87,22.12,28.92,5254.0,1882.0,459.0,22988.0,60600,448
2020-05,347687.0,5.234270e+05,709437.0,1158.0,7137.0,995.0,901.0,1398.0,149.0,161.0,...,26972.0,8.44,24.68,45.25,1279.0,545.0,106.0,13145.0,62620,597
2020-06,81872.0,1.286400e+05,194299.0,163.0,3851.0,154.0,207.0,233.0,47.0,50.0,...,18677.0,12.50,28.13,51.72,169.0,68.0,36.0,2826.0,60600,710
2020-07,114015.0,1.399699e+05,231358.0,59.0,4184.0,83.0,166.0,522.0,31.0,40.0,...,18141.0,34.95,40.55,46.40,58.0,24.0,15.0,720.0,62620,868
2020-08,258720.0,3.573637e+05,490937.0,75.0,16339.0,246.0,471.0,580.0,92.0,97.0,...,19087.0,31.10,33.78,36.46,227.0,51.0,52.0,1047.0,62620,1005
2020-09,1615343.0,1.834558e+06,2632893.0,187.0,50008.0,577.0,1694.0,1393.0,247.0,255.0,...,19892.0,36.17,37.59,39.07,1022.0,138.0,163.0,2708.0,60600,1104
2020-10,3357599.0,4.486952e+06,5773497.0,991.0,229558.0,2353.0,7080.0,6800.0,1165.0,1138.0,...,26366.0,30.81,31.51,32.20,5371.0,714.0,584.0,11624.0,62620,1275


In [22]:
#print('dataframe for RIVM data created, per month per municipality: ririvm_month_municipalityvm_per_month')
#rivm_month_municipality

#### RIVM per year

In [23]:
rivm_per_year = rivm_per_day.groupby(rivm_per_day.index.year).sum()
rivm_per_year.index.rename('year', inplace=True)
rivm_per_year.index = pd.PeriodIndex(rivm_per_year.index, freq='Y')
rivm_per_year = rivm_per_year[rivm_per_year.index < population_per_year.index.max()]

print('dataframe for RIVM data created, per month: rivm_per_year')
rivm_per_year

dataframe for RIVM data created, per month: rivm_per_year


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
year,,,,,,,,,,,,,,,,,,,,,
2020,15038960.0,2.013282e+07,26119189.0,11405.0,795545.0,20036.0,35214.0,34563.0,6577.0,6534.0,...,210074.0,274.22,343.15,428.52,29286.0,6096.0,3048.0,99516.0,644380,9343


#### RIVM from weekreports: per week per agegroup

In [24]:
def add_data_from(f):    
    def merge_split(df):
        df_temp = df[df.columns].apply(
            lambda x: ' '.join(x.dropna().astype(str)),
            axis=1
        )
        df_temp = pd.DataFrame(df_temp.str.split(expand=True))
        return df_temp
    
    #if debug == untill:
    #    print(filename)
    #    print(debug)
    df = pd.read_csv(rivm_pdf_data_loc+f)
    # selecting usable rows

    if debug == untill:
        print(df.iloc[1,0])
    if df.iloc[0,0] == 'Groep Aantal positief Aantal getest Percentage positief':
        text_in_column = 'Tabel 12: Aantal testen en percentage positief per doelgroep in de afgelopen twee weken1.'
        if (text_in_column in df.iloc[:,0].values) == False:
            text_in_column = 'Tabel 13: Aantal testen en percentage positief per doelgroep in de afgelopen week1.'
        df = df.iloc[df.index[df.index[df.iloc[:,0] == text_in_column].values[0]:,...]]
    
    if df.columns[0] != 'Leeftijdsgroep':
        df = df[df.iloc[:,0].str.contains('jaar')]
    df.iloc[:,0].replace({'jaar2':'jaar', '70\+':'70 en ouder', '80\+':'80 en ouder'}, regex=True, inplace=True)
    
    # setting Date to index
    date = [int(i) for i in df.iloc[0,-1].split('-')]
    date = dt(date[0], date[1], date[2])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Week'] = df['Date'].dt.isocalendar().week
    df.set_index('Date', inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    if debug == untill:
        print(date)
    
    # setting Agegroup
    df_temp = merge_split(df)
    if '-' in df_temp.iloc[0,0]:
        df_temp2 = df_temp.iloc[:,0].str.split('-',expand=True)
        df_temp2['Agegroup'] = df_temp2.iloc[:,0]+' - '+df_temp2.iloc[:,1]
        if '95+' in df_temp2.iloc[-1,0]:
            df_temp2.iloc[-1,-1] = '95 en ouder'
        df_temp.iloc[:,0] = df_temp2['Agegroup']
        df_temp = merge_split(df_temp)
    if debug == untill:
        print(df_temp)
    
    df['Agegroup'] = df_temp.iloc[:,0]+' '+df_temp.iloc[:,1]+' '+df_temp.iloc[:,2]
    # setting values
    #print(df_temp)
    if date < dt(2020, 12, 1) :
        df['Tested positive'] = df_temp.iloc[:,-4]
        df['Total tested'] = df_temp.iloc[:,-3]
    elif date == dt(2020, 12, 1) :
        df['Tested positive'] = df_temp.iloc[:,3].astype(int) + df_temp.iloc[:,6].astype(int)
        df['Total tested'] = df_temp.iloc[:,4].astype(int) + df_temp.iloc[:,7].astype(int)
    elif date >= dt(2021, 4, 27) :
        df['Tested positive'] = df_temp.iloc[:,4]
        df['Total tested'] = df_temp.iloc[:,3]
        df['Tested positive'] = round(df['Tested positive'].astype(float) / 100 * df['Total tested'].astype(float),0)
        df['Tested positive'] = df['Tested positive'].astype(int)
        if len(df[df['Agegroup'].str.find('en ouder') > -1]) > 1:
            for index, value in df['Agegroup'].reset_index(drop=True).items():
                if value.find('en ouder') > -1:
                    i = index
                    break
            df = df.iloc[:i+1,:]
    else:
        df['Tested positive'] = df_temp.iloc[:,-3]
        df['Total tested'] = df_temp.iloc[:,-4]
    df['cat'] = filename.find('Leeftijd')
    df['cat'].replace({-1: 'Groep', 14: 'Leeftijd', 16: 'Leeftijd', 20: 'Leeftijd', 22: 'Leeftijd', 25: 'Leeftijd', 30: 'Leeftijd'}, inplace=True)
    
    df = df[['Week', 'Agegroup', 'Tested positive', 'Total tested', 'cat']]
    if debug == untill:
        print(df)
    return df

In [25]:
## create df
debug = 0
untill = 500

# add dummy csv if no table is found in pdf
global filelist
filelist = os.listdir(rivm_pdf_data_loc)
filelist.sort()
dates = [x[:13] for x in filelist]
dates = {key:len(list(group)) for key, group in groupby(dates)}
for date, count in dates.items():
    if count == 1:
        if date != '.ipynb_checkp':
            dummy = pd.DataFrame()
            dummy.to_csv(rivm_pdf_data_loc+date+'-dummy.csv', index=False)

rivm_week_agegroup = pd.DataFrame()

for filename in filelist:
    if debug == untill+1:
        break
    if '.csv' in filename:
        #print(debug)
        if filename[-10:] == '-dummy.csv' or (dates[filename[:13]] > 2 and filename[-9:-4] == 'Groep'):
            #print(filename+' skipping')
            pass
        else:
            #print(filename)
            rivm_week_agegroup = pd.concat([rivm_week_agegroup,add_data_from(filename)])
            #print( data.loc['2021-04-27':'2021-04-27',] )#.groupby('Date').nunique() )

        if debug == untill:
            #print(data)
            break
        debug += 1

rivm_week_agegroup.index = pd.DatetimeIndex(rivm_week_agegroup.index)
rivm_week_agegroup['Week'] = rivm_week_agegroup.index.year * 100 + rivm_week_agegroup['Week']
rivm_week_agegroup['Tested positive'] = rivm_week_agegroup['Tested positive'].fillna(0).astype(int)
rivm_week_agegroup['Total tested'] = rivm_week_agegroup['Total tested'].fillna(0).astype(int)

data_temp = rivm_week_agegroup['cat'].reset_index().drop_duplicates().groupby('Date').count().astype(int)
data_temp.rename(columns={'cat':'count'}, inplace=True)
rivm_week_agegroup = pd.merge(rivm_week_agegroup,data_temp, on='Date')
rivm_week_agegroup['cat_count'] = rivm_week_agegroup['cat'].astype(str)+rivm_week_agegroup['count'].astype(str)
rivm_week_agegroup = rivm_week_agegroup[rivm_week_agegroup['cat_count'] != 'Groep2']
rivm_week_agegroup.drop(columns=['cat', 'count', 'cat_count'], inplace=True)

In [26]:
rivm_week_agegroup['Agegroup'].replace({
    'Kinderen 0-3 jaar': '0 - 3', 
    'Kinderen 0-6 jaar': '0 - 6', 
    'Kinderen 4-11 jaar': '4 - 11', 
    'Kinderen 4-11 jaar3': '4 - 11', 
    'Kinderen 4-12 jaar': '4 - 12', 
    'Kinderen 7-12 jaar': '7 - 12', 
    'Kinderen 12-17 jaar': '12 - 17',
    'Kinderen 12-17 jaar4': '12 - 17',
    'Kinderen 13-17 jaar': '13 - 17',
    'Kinderen 13-18 jaar': '13 - 18', 
    'Jongvolwassenen 18-24 jaar': '18 - 24'}, inplace=True)

rivm_week_agegroup['Group'] = rivm_week_agegroup['Agegroup'].copy()
rivm_week_agegroup.loc[:,'Group'].replace({
    '0 - 3': 'Kinderen', 
    '0 - 4': 'Kinderen',
    '0 - 6': 'Kinderen', 
    '0 - 12': 'Basisschool',
    '5 - 9': 'Basisschool',
    '10 - 14': 'Basisschool',
    '4 - 12': 'Basisschool', 
    '7 - 12': 'Basisschool', 
    '12 - 17': 'Pubers',
    '13 - 17': 'Pubers',
    '13 - 18': 'Pubers', 
    '15 - 19': 'Pubers', 
    '18 - 24': 'Jongvolwassenen',
    '20 - 24': 'Jongvolwassenen', 
    '25 - 29': 'Jongvolwassenen',
    '30 - 34': 'Volwassenen',
    '30 - 39': 'Volwassenen', 
    '35 - 39': 'Volwassenen',
    '40 - 44': 'Volwassenen',
    '40 - 49': 'Volwassenen',
    '45 - 49': 'Volwassenen',
    '50 - 54': 'Volwassenen',
    '50 - 59': 'Volwassenen',
    '55 - 59': 'Volwassenen',
    '60 - 64': 'Volwassenen',
    '60 - 69': 'Volwassenen', 
    '65 - 69': 'Volwassenen',
    '70 en ouder': '70 en ouder',
    '70 - 74': '70 en ouder',
    '70 - 79': '70 en ouder', 
    '75 - 79': '70 en ouder',
    '80 en ouder': '70 en ouder',
    '80 - 84': '70 en ouder',
    '85 - 89': '70 en ouder',
    '90 - 94': '70 en ouder'}, inplace=True)

rivm_week_agegroup['Agerange'] = rivm_week_agegroup['Agegroup'].copy()
rivm_week_agegroup.loc[:,'Agerange'].replace({
    '0 - 3': '0 - 12', 
    '0 - 4': '0 - 12',
    '0 - 6': '0 - 12', 
    '0 - 12': '0 - 12',
    '5 - 9': '0 - 12',
    '10 - 14': 'groep 7/8 + brugklas',
    '4 - 12': '0 - 12', 
    '7 - 12': '0 - 12', 
    '12 - 17': '13 - 24',
    '13 - 17': '13 - 24',
    '13 - 18': '13 - 24', 
    '15 - 19': '13 - 24', 
    '18 - 24': '13 - 24',
    '20 - 24': '13 - 24', 
    '25 - 29': '25 - 39',
    '30 - 34': '25 - 39',
    '30 - 39': '25 - 39', 
    '35 - 39': '25 - 39',
    '40 - 44': '40 - 49',
    '40 - 49': '40 - 49',
    '45 - 49': '40 - 49',
    '50 - 54': '50 - 59',
    '50 - 59': '50 - 59',
    '55 - 59': '50 - 59',
    '60 - 64': '60 - 69',
    '60 - 69': '60 - 69', 
    '65 - 69': '60 - 69',
    '70 en ouder': '70 en ouder',
    '70 - 74': '70 en ouder',
    '70 - 79': '70 en ouder', 
    '75 - 79': '70 en ouder',
    '80 en ouder': '70 en ouder',
    '80 - 84': '70 en ouder',
    '85 - 89': '70 en ouder',
    '90 - 94': '70 en ouder',
    '95 en ouder': '70 en ouder'}, inplace=True)

In [27]:
#calculate percentage
rivm_week_agegroup['Percentage'] = round(rivm_week_agegroup['Tested positive'] / rivm_week_agegroup['Total tested'] * 100, 3)

In [28]:
print('dataframe from cbs created, per week per agegroup: rivm_week_agegroup')
rivm_week_agegroup.to_csv(tmp, index=False)
rivm_week_agegroup

dataframe from cbs created, per week per agegroup: rivm_week_agegroup


,Week,Agegroup,Tested positive,Total tested,Group,Agerange,Percentage
Date,,,,,,,
2020-07-07,202028,0 - 6,25,7033,Kinderen,0 - 12,0.355
2020-07-07,202028,7 - 12,50,19446,Basisschool,0 - 12,0.257
2020-07-07,202028,13 - 18,97,8661,Pubers,13 - 24,1.120
2020-07-21,202030,0 - 6,5,898,Kinderen,0 - 12,0.557
2020-07-21,202030,7 - 12,23,6221,Basisschool,0 - 12,0.370
...,...,...,...,...,...,...,...
2022-01-18,202203,40 - 49,34612,95087,Volwassenen,40 - 49,36.400
2022-01-18,202203,50 - 59,30503,99037,Volwassenen,50 - 59,30.800
2022-01-18,202203,60 - 69,11061,41581,Volwassenen,60 - 69,26.601


In [29]:
rivm_week_agegroup.to_csv(tmp, index=False)

#### RIVM per month per municipality

In [30]:
#print('dataframe for RIVM data created, per month per municipality: rivm_month_municipality')
#rivm_month_municipality

## Merging CBS and RIVM
### Per week deceased_per_week + rivm_per_week

In [31]:
per_week = pd.merge(deceased_per_week, rivm_per_week, left_index=True, right_index=True, how='outer', sort=True)

print('dataframe CBS and RIVM combined, per week: per_week')
per_week

dataframe CBS and RIVM combined, per week: per_week


Deceased  prev_low       prev_avg  prev_up  Deceased_covid  \
year week                                                               
1995 0        394.0       NaN            NaN      NaN             NaN   
     1       2719.0       NaN            NaN      NaN             NaN   
     2       2823.0       NaN            NaN      NaN             NaN   
     3       2609.0       NaN            NaN      NaN             NaN   
     4       2664.0       NaN            NaN      NaN             NaN   
...             ...       ...            ...      ...             ...   
2021 52      2698.0       0.0  478609.508399      0.0             0.0   
2022 0        946.0       0.0  182405.314442      0.0             0.0   
     1       3392.0       0.0  572265.469755      0.0             0.0   
     2       3238.0       0.0  494964.869298      0.0             0.0   
     3          NaN       0.0  368598.186145      0.0             0.0   

           Positive_tests  Hospital_admission  Hospital_admission_NICE  \
year week                                                                
1995 0                NaN                 NaN                      NaN   
     1                NaN                 NaN                      NaN   
     2                NaN                 NaN                      NaN   
     3                NaN                 NaN                      NaN   
     4                NaN                 NaN                      NaN   
...                   ...                 ...                      ...   
2021 52               0.0                 0.0                    835.0   
2022 0                0.0                 0.0                    217.0   
     1                0.0                 0.0                    951.0   
     2                0.0                 0.0                    732.0   
     3                0.0                 0.0                    547.0   

           Hospital_admission_NICE_notification  IC_admission  ...  \
year week                                                      ...   
1995 0                                      NaN           NaN  ...   
     1                                      NaN           NaN  ...   
     2                                      NaN           NaN  ...   
     3                                      NaN           NaN  ...   
     4                                      NaN           NaN  ...   
...                                         ...           ...  ...   
2021 52                                   954.0         125.0  ...   
2022 0                                    170.0          45.0  ...   
     1                                    943.0         146.0  ...   
     2                                    792.0          97.0  ...   
     3                                    783.0          49.0  ...   

           IC_Nieuwe_Opnames_COVID  Kliniek_Nieuwe_Opnames_COVID  \
year week                                                          
1995 0                         NaN                           NaN   
     1                         NaN                           NaN   
     2                         NaN                           NaN   
     3                         NaN                           NaN   
     4                         NaN                           NaN   
...                            ...                           ...   
2021 52                        0.0                           0.0   
2022 0                         0.0                           0.0   
     1                         0.0                           0.0   
     2                         0.0                           0.0   
     3                         0.0                           0.0   

           IC_Bedden_TOTAAL  Rt_low  Rt_avg  Rt_up  \
year week                                            
1995 0                  NaN     NaN     NaN    NaN   
     1                  NaN     NaN     NaN    NaN   
     2                  NaN     NaN     NaN    NaN   
     3                  NaN     

In [32]:
#print('dataframe CBS and RIVM combined, per week per municipality: week_municipality')
#week_municipality

### Per month population_per_month + rivm_per_month

In [33]:
per_month = pd.merge(population_per_month, rivm_per_month, on='Y-M', how='outer', sort=True)

print('dataframe CBS and RIVM combined, per month: per_month')
per_month

#print('dataframe CBS and RIVM combined, per month per municipality: month_municipality')
#month_municipality

dataframe CBS and RIVM combined, per month: per_month


,Born,Deceased,Population,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
Y-M,,,,,,,,,,,,,,,,,,,,,
2002-01,17019.0,13469.0,16112900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02,15448.0,11735.0,16118953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-03,16792.0,13281.0,16124125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-04,15995.0,11968.0,16130246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-05,16800.0,11623.0,16136807.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07,16053.0,12557.0,17515152.0,10910.0,7.747672e+05,77606.0,82.0,180361.0,470.0,1497.0,...,17343.0,34.11,35.03,35.99,592.0,41.0,151.0,1923.0,62651.0,872.0
2021-08,15798.0,12726.0,17535520.0,0.0,1.221315e+06,0.0,184.0,75655.0,816.0,1957.0,...,0.0,29.61,30.82,32.05,832.0,100.0,122.0,4883.0,62651.0,1010.0
2021-09,16086.0,12702.0,17565777.0,0.0,8.791437e+05,0.0,160.0,62120.0,640.0,1395.0,...,0.0,28.26,29.59,30.93,478.0,47.0,113.0,3883.0,60630.0,1108.0


### Per month population_per_year + rivm_per_year

In [34]:
per_year = pd.merge(population_per_year.drop_duplicates(subset=['Deceased'], keep='last'), rivm_per_year, left_index=True, right_index=True, how='outer', sort=True, copy=False, validate='m:1')
per_year = per_year[(per_year['Born'].isnull() * 1 + (1 - per_year['prev_low'].isnull() * 1)) != 2]

print('dataframe CBS and RIVM combined, per month: per_year')
per_year

#print('dataframe CBS and RIVM combined, per year per municipality: year_municipality')
#year_municipality

dataframe CBS and RIVM combined, per month: per_year


,Deceased,Born,Population,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,...,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes,year,week
year,,,,,,,,,,,,,,,,,,,,,
1995,135675.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,137561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,135783.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,137482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999,140487.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,140527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,140377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,142355.0,202083.0,193809414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,141936.0,200297.0,194705905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## pandas_profiling